# Sattelite Image Analysis


Aria Burk
u1102905@utah.edu
1102905

Joseph Cochran 
u1193064@utah.edu
1193064

John Simonyan
u0969548@utah.edu
0969548

**Background and Motivation:**

As a group, each of us took an interest in NASA’s public APIs and the possibilities of implementing data science within the context of geoscience. Satellite imagery collected in near real-time all across the globe can be quite valuable for those who want to better understand weather patterns, geographical changes over time, disaster prediction/prevention, and many other large-scale surveys.

Sattelite image anaylsis can have a plethora of pracitcal applicatons. Because sattelites can take pictures all over the surface of the planet they can capture formations and patterns in ways we can't see on the ground. In 2019 a group of researchers trained an ai to find geoglyphs, discovering new sets of Nazca lines (Becky Little,https://www.history.com/news/nazca-lines-peru-ai). This just shows one example of the potential of large scale sattelite image analysis. 


**Project Objectives:**

Listed below are some flexible objectives that we can expand on or dismiss during our project’s exploratory phase.
Explore the metadata collected and associated with each image to find meaningful correlations between different labels and geographical coordinates or other identifiers.
    
- Analyze select images and develop an algorithm that determines whether the images are similar enough to contain the same subject or location.

- Inversely, find different images of the same subject or location, test the previous algorithm’s accuracy, and compare it with different methodologies.
- Test datasets against other tracked events to find correlations and potential patterns
    
- Train models on datasets to make predictions on a set of qualifiers based on the datasets


**Data:**

By querying the "Earth" NASA API at https://api.nasa.gov/planetary/earth/imagery, we receive a base64 encoded png image that was taken by satellite. 

The query inputs are longitude, latitude, dimensions, and date the image was taken.

Geo.json dataset containing latitude and longitude along with other attributes, city, country. Using these patameters we can more clearly classify the queried data.

Cross referencing our image queries with locational queries to find relevent metadata to apply to our images.
Metadata exploratory analysis can ecpand dataset very quickly.

TODO:
1. Specify what dataset we want(The geojson I found is of u.s. cities ) I.e which cities/dates to pull from
2. iterate through geoJson making api calls with every iteration
3. Time out iteration to not overload api. (1,000 requests per hour)
4. save response data to files as png with appropriate classifiers to categorize images appropriately
5. write out method to parse images to histogram or heat map
6. plot out color ranges and heatmap ranges for all the images
7. analyize data ranges to make some predictions or inferences
8. Generate appropriate dataframes with images and img metadata for an easy visualization
    

In [1]:
#load csv of city coordinates
import pandas as pd
from IPython.core.display import HTML

us_cities = pd.read_csv("us_cities_v2.csv")
us_cities

,city,state_id,state,county,lat,long,population,density,source,military,incorporated,capital,timezone,ranking,zips
0,New York,NY,New York,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,False,America/New_York,1,309
1,Los Angeles,CA,California,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,False,America/Los_Angeles,1,196
2,Chicago,IL,Illinois,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,False,America/Chicago,1,85
3,Miami,FL,Florida,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,False,America/New_York,1,30
4,Dallas,TX,Texas,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,False,America/Chicago,1,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19086,Portage Creek,AK,Alaska,Dillingham,58.9051,-157.6695,3,0,polygon,False,False,False,America/Anchorage,3,1
19087,Gross,NE,Nebraska,Boyd,42.9461,-98.5697,2,6,polygon,False,True,False,America/Chicago,3,1
19088,Lotsee,OK,Oklahoma,Tulsa,36.1334,-96.2091,2,39,polygon,False,True,False,America/Chicago,3,1
19089,The Ranch,MN,Minnesota,Mahnomen,47.3198,-95.6952,2,2,polygon,False,True,False,America/Chicago,3,1


In [2]:
import requests
import base64
import imageio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np 
import os

from PIL import Image
from io import BytesIO

#example query
# url = "https://api.nasa.gov/planetary/earth/imagery?lon=-111.9306&lat=40.7777&dim=.2&date=2020-04-26&api_key=II3XUxmGYfFKmyapNeXjrH3Os3ILNhQScjW6bbMT"
# response = requests.get(url)

#image returned 
# i = Image.open(BytesIO(response.content))
# i

#images is saved to file with city name and date taken as format
# i.save('slc_2020_04_26.png')

image_test = Image.open('slc_2020_04_26.png')

#img to histogram
img_hst = image_test.histogram()

#seperate rgb values from histogram to plot seperately
def getRed(R): 
    return '#%02x%02x%02x'%(R,0,0)
def getGreen(G): 
    return '#%02x%02x%02x'%(0,G,0)
def getBlue(B):
    return '#%02x%02x%02x'%(0,0,B)

#iterate through img histogram to find rgb values
Red=img_hst[0:256] # indicates Red
Green=img_hst[256:512] # indicated Green
Blue=img_hst[512:768] # indicates Blue

for i in range(0, 256):
    plt.bar(i, Red[i], color = getRed(i),alpha=1)
    plt.figure(1) # plot red
for i in range(0, 256):
    plt.bar(i, Green[i], color = getGreen(i),alpha=0.3)
    plt.figure(2) # plot green
for i in range(0, 256):
    plt.bar(i, Blue[i], color = getBlue(i),alpha=0.3)
    plt.figure(3) #plot blue
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'slc_2020_04_26.png'

In [ ]:
# Method for data cleaning and preparation:

# import pandas as pd

# us_cities = pd.read_csv("uscities_clean.csv")
# us_cities = us_cities.drop_duplicates(subset='city', keep='first')
# us_cities = us_cities.reset_index(drop=True)
# #19,091 cities stored

# states = pd.read_csv("state_capitals.csv")
# capitals = [_ for _ in states['Capital']]

# is_capital = [city in capitals for city in us_cities['city']]
# us_cities['capital'] = is_capital

# us_cities['zips'] = [len(str.split(str(_))) for _ in us_cities['zips']]
# us_cities

# Manually included Washington DC as (quasi-)"State Capital"
# Corrected 'St. Paul' to 'Saint Paul' to be detected by script

# us_cities.to_csv(r'...\SatteliteAnalysis\us_cities_v2.csv', index = False)

**Ethical considerations:**

Regarding our datasets, there is an uncertain degree of bias that affects the process of taking satellite imagery. Perhaps some regions are more “photogenic” or are more likely to have specific documented events such as natural disasters or the like. Considering this, we should be mindful that our interpretations may be influenced by imaging bias or analytical bias. If a similar analysis to this project was used for developing maps or for disaster prevention efforts, there could be a wide range of consequences.


**Data Processing:**

Utilizing geo.json coordinates to load parameters for api queries. By using different location sets we can find significant locations to test. By normalizing the query data we can train a test model to recognize certain types of images based on different parameters. Parameters can be based on expected attributes of trained datasets.

As we collect our data, we will need to process images, sort visual findings, and store our conclusions. We plan to use built-in python packages to visualize our results (i.e., see trends in data, and portray model fitments). These packages include, but are not limited to, NumPy, SciPy, SciKit Learn, Matplotlib, and NASA’s Bingo. Data Processing will be the crux of this project and allow us to come to many conclusions about weather patterns, climate change, and other geographic changes that we might see. Skills we have learned throughout this semester will aid in these tasks.

Allocating the appropriate metadata to allow us to make hypothetical predictions about changes occurring due to features outside of the received information. By categorizing locational data appropriately, we can classify regions by qualities outside of just its specific location. The features found in regions with rockier terrain or in areas affected by changes in temperature are all valuable classifiers that we can make predictions and base models on.

In [ ]:
#example query from a previous date
url = "https://api.nasa.gov/planetary/earth/imagery?lon=111.10&lat=39.01&dim=.25&date=2019-03-26&api_key=II3XUxmGYfFKmyapNeXjrH3Os3ILNhQScjW6bbMT"
response = requests.get(url)

#image returned for comparison 
i2 = Image.open(BytesIO(response.content))

#plotted and by using our dimendions we can measure distances and scale accurately
imgplot = plt.imshow(i2)
plt.show()

**Exploratory Analysis:**

As outlined in our Project Objectives, metadata associated with each image may be an essential way to navigate, categorize, and classify the items within our dataset. Doing preliminary analysis with the metadata might reveal relationships we want to explore further, deter us from other approaches, and act as a foundation for any further studies involved with image processing. 
    
By taking image data from the same location over a given period of time, we will be able to identify changes in location regardless of any temporal factors. Pixel analysis to a heat map can show change in color, or with the right masking tools change in geographical features and or cloud coverage. 


**Analysis Methodology:**

Cross references our datasets against normalized test data, to ensure results our results are within the expected range. Sampling random queries can allow us to double check and validate before we make any assertions based on false data processing. We want to ensure as much accuracy as possible, by allocating the appropriate stopgaps in our data analysis we can maintain some control over any potential analysis inaccuracies.



**Project Schedule:**
- March 26th - Project Proposal Due
- April 6th - Target for Cleaned/Explored Dataset
- April 11th - Project Milestone Due
- April 19th - Target for Completed Screen-Cast Presentation
- April 25th - Final Project Submission

**Sources:**
Geo.json of U.S. cities containing population, location, and more features: https://simplemaps.com/data/us-cities
.csv containing U.S. state capitals and state features: https://www.downloadexcelfiles.com/us_en/download-excel-file-list-state-capitals-united-states